<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Deploy Naas Cloud

**Description:** This notebook allows you to run your data product on your Naas server.

## Input

### Import libraries

In [ ]:
import naas
import os
import glob
import naas_data_product

### Setup variables
- `cron`: Represents the scheduling pattern of your notebook (https://crontab.guru/)
- `pipeline_path`: Main pipeline to be scheduled in production
- `delete`: Boolean to be used as True to removed scheduler and dependencies

In [ ]:
cron = "0 12 * * *"
pipeline_path = "__pipeline__.ipynb"
delete = False

## Model

### Add or delete scheduler main pipeline

In [ ]:
# Schedule pipeline
if not delete:
    naas.scheduler.add(pipeline_path, cron=cron)
    print("⏰ Scheduler:", cron)
else:
    naas.scheduler.delete(pipeline_path)

### Add pipeline templates to dependencies

In [ ]:
engines = [
    "content-engine",
    "growth-engine",
    "sales-engine",
    "operations-engine",
    "finance-engine",
    "opendata-engine",
]
for engine in engines:
    # Engine model directory
    engine_model_dir = os.path.join(naas_data_product.MODELS_PATH, engine)

    # Add dependencies
    files = (
        glob.glob(os.path.join(engine_model_dir, "core", "domain", "*.ipynb")) + 
        glob.glob(os.path.join(engine_model_dir, "custom", "*.ipynb")) + 
        [os.path.join(engine_model_dir, "__pipeline__.ipynb")] + 
        [os.path.join(engine_model_dir, "__plugin__.ipynb")]
    )
    for file in files:
        if not delete:
            naas.dependency.add(file)
            print("🔗 Dependency:", file)
            print()
        else:
            naas.dependency.delete(file)

### Add utils to dependencies

In [ ]:
files = glob.glob(f"{naas_data_product.UTILS_PATH}/*.ipynb")

for file in files:
    if not delete:
        naas.dependency.add(file)
        print("🔗 Dependency:", file)
        print()
    else:
        naas.dependency.delete(file)

## Output

### Display result

In [ ]:
if not delete:
    print("✅ Scheduler and dependencies sent to production")
else:
    print("❎ Scheduler and dependencies removed from production")